In [6]:
import requests, json
import pandas as pd

#https://centadata.publicdata.hk/centadata-54990be.json?sql=select+*+from+centadataall+limit+10
base_url = 'https://centadata.publicdata.hk/centadata-54990be.json'
# params_ini = {
#     'apiKey': api_key
# }

params = {
    'sql': 'select * from centadataall limit 10'
}

data = requests.get( base_url, params = params)

def GetJson(url, params):
    data = requests.get(url, params = params)
    if data.status_code == 200:
        return json.loads(data.text)
    else:
        return None

In [14]:
cdata = json.loads( data.text )
df = pd.DataFrame(data = cdata['rows'],
                 columns = cdata['columns']
                 )

In [15]:
df.head()

,unitcode,txdate,price,buildingcode,saleablearea,grossarea,xaxis,yaxis,buildingname,phasecode,...,buildingnamechinese,phasenamechinese,estatenamechinese,buildingaddresschinese,region,regionchinese,regioncode,propertydeveloperchinese,propertyfacilitieschinese,propertymanagerchinese
0,MMMLITSATMTT,2012-03-22,1780000,MMMLIHSAHM,392,NaN,NO. 205A,1/F,PAK TUNG BUILDING,None,...,百通大樓,,,通菜街205A號,Mongkok/Yaumatei,旺角/油麻地,203,None,None,None
1,SYSKPYGXWEWW,2002-10-07,24000000,SYSKPPGXWE,2894,3319.0,HOUSE 9,NO. 62 - 66,HOUSE 9,SYSKPPGXPE,...,9座,ONDINA HEIGHTS,,山頂道62-66號,Peak/South,山頂/南區,107,None,None,None
2,SYSKPYGXWEWW,1997-07-08,69700000,SYSKPPGXWE,2894,3319.0,HOUSE 9,NO. 62 - 66,HOUSE 9,SYSKPPGXPE,...,9座,ONDINA HEIGHTS,,山頂道62-66號,Peak/South,山頂/南區,107,None,None,None
3,EKSYGSDSGSGG,1996-12-24,2590000,EKSYGPDSPS,927,NaN,NO. 34,G/F,MAN SAU CHUNG BUILDING,None,...,萬壽松大廈,,,盛德街34號,To Kwa Wan,土瓜灣,206,None,None,None
4,MMMLIMSAMMMM,2017-02-16,2880000,MMMLIHSAHM,392,NaN,NO. 205A,2/F,PAK TUNG BUILDING,None,...,百通大樓,,,通菜街205A號,Mongkok/Yaumatei,旺角/油麻地,203,None,None,None


In [16]:
import requests, json
import pandas as pd
base_url = 'https://centadata.publicdata.hk/centadata-54990be.json'
params = {
    'sql': 'select * from centadataall limit 10'
}
data = requests.get( base_url, params = params)
cdata = json.loads( data.text )
df = pd.DataFrame(data = cdata['rows'],
                 columns = cdata['columns']
                 )
df.head()

,unitcode,txdate,price,buildingcode,saleablearea,grossarea,xaxis,yaxis,buildingname,phasecode,...,buildingnamechinese,phasenamechinese,estatenamechinese,buildingaddresschinese,region,regionchinese,regioncode,propertydeveloperchinese,propertyfacilitieschinese,propertymanagerchinese
0,MMMLITSATMTT,2012-03-22,1780000,MMMLIHSAHM,392,NaN,NO. 205A,1/F,PAK TUNG BUILDING,None,...,百通大樓,,,通菜街205A號,Mongkok/Yaumatei,旺角/油麻地,203,None,None,None
1,SYSKPYGXWEWW,2002-10-07,24000000,SYSKPPGXWE,2894,3319.0,HOUSE 9,NO. 62 - 66,HOUSE 9,SYSKPPGXPE,...,9座,ONDINA HEIGHTS,,山頂道62-66號,Peak/South,山頂/南區,107,None,None,None
2,SYSKPYGXWEWW,1997-07-08,69700000,SYSKPPGXWE,2894,3319.0,HOUSE 9,NO. 62 - 66,HOUSE 9,SYSKPPGXPE,...,9座,ONDINA HEIGHTS,,山頂道62-66號,Peak/South,山頂/南區,107,None,None,None
3,EKSYGSDSGSGG,1996-12-24,2590000,EKSYGPDSPS,927,NaN,NO. 34,G/F,MAN SAU CHUNG BUILDING,None,...,萬壽松大廈,,,盛德街34號,To Kwa Wan,土瓜灣,206,None,None,None
4,MMMLIMSAMMMM,2017-02-16,2880000,MMMLIHSAHM,392,NaN,NO. 205A,2/F,PAK TUNG BUILDING,None,...,百通大樓,,,通菜街205A號,Mongkok/Yaumatei,旺角/油麻地,203,None,None,None
